Perfect! Let’s create a **complete Appointment Booking Bot** using **RASA**. This bot will allow users to:

* Book appointments
* Check existing appointments
* Cancel appointments

We’ll cover **intents, entities, slots, custom actions, stories, domain, and deployment setup**.

---

# 📅 **Appointment Booking Bot using RASA**

---

## **1️⃣ Project Overview**

**Objective:**
Build a conversational bot to manage appointments for a clinic, salon, or office.

**Example Conversation:**

```
User: I want to book an appointment
Bot: Sure! Which date would you like?
User: Tomorrow at 3 PM
Bot: Great! Your appointment is booked for tomorrow at 3 PM.
User: Cancel my appointment
Bot: Your appointment has been canceled.
```

---

## **2️⃣ Folder Structure**

```
appointment_bot/
│
├── actions/
│   └── actions.py
├── data/
│   ├── nlu.yml
│   ├── stories.yml
│   └── rules.yml
├── config.yml
├── domain.yml
├── credentials.yml
├── endpoints.yml
└── requirements.txt
```

---

## **3️⃣ Install Dependencies**

**File:** `requirements.txt`

```
rasa
rasa[spacy]
spacy
```

Install dependencies:

```bash
python -m venv rasa_env
source rasa_env/bin/activate   # Mac/Linux
rasa_env\Scripts\activate      # Windows

pip install -r requirements.txt
python -m spacy download en_core_web_sm
```

---

## **4️⃣ Define Intents & Entities**

**File:** `data/nlu.yml`

```yaml
version: "3.1"

nlu:
- intent: greet
  examples: |
    - hi
    - hello
    - good morning

- intent: goodbye
  examples: |
    - bye
    - see you
    - goodbye

- intent: book_appointment
  examples: |
    - I want to book an appointment
    - Can I book a slot?
    - Schedule an appointment
    - I need a meeting tomorrow

- intent: cancel_appointment
  examples: |
    - Cancel my appointment
    - I want to cancel my booking
    - Delete my appointment

- intent: check_appointment
  examples: |
    - Do I have an appointment?
    - Check my booking
    - Show my appointment

entities:
  - date
  - time
```

---

## **5️⃣ Define Domain & Responses**

**File:** `domain.yml`

```yaml
version: "3.1"

intents:
  - greet
  - goodbye
  - book_appointment
  - cancel_appointment
  - check_appointment

entities:
  - date
  - time

slots:
  date:
    type: text
  time:
    type: text

responses:
  utter_greet:
    - text: "Hi! I can help you book, check, or cancel appointments."

  utter_goodbye:
    - text: "Goodbye! Have a great day."

  utter_ask_date:
    - text: "Which date would you like to book the appointment for?"

  utter_ask_time:
    - text: "What time would you prefer?"

  utter_confirm_booking:
    - text: "Your appointment is booked for {date} at {time}."

  utter_cancel_confirmation:
    - text: "Your appointment has been canceled."

  utter_no_appointment:
    - text: "You have no appointment currently."
    
actions:
  - action_book_appointment
  - action_cancel_appointment
  - action_check_appointment
```

---

## **6️⃣ Create Stories**

**File:** `data/stories.yml`

```yaml
version: "3.1"

stories:
- story: book appointment
  steps:
  - intent: greet
  - action: utter_greet
  - intent: book_appointment
  - action: utter_ask_date
  - action: action_book_appointment
  - intent: goodbye
  - action: utter_goodbye

- story: cancel appointment
  steps:
  - intent: cancel_appointment
  - action: action_cancel_appointment
  - intent: goodbye
  - action: utter_goodbye

- story: check appointment
  steps:
  - intent: check_appointment
  - action: action_check_appointment
  - intent: goodbye
  - action: utter_goodbye
```

---

## **7️⃣ Pipeline & Policies**

**File:** `config.yml`

```yaml
language: en

pipeline:
  - name: WhitespaceTokenizer
  - name: CountVectorsFeaturizer
  - name: DIETClassifier
  - name: EntitySynonymMapper
  - name: ResponseSelector

policies:
  - name: MemoizationPolicy
  - name: RulePolicy
  - name: TEDPolicy
```

---

## **8️⃣ Custom Actions**

**File:** `actions/actions.py`

```python
from rasa_sdk import Action, Tracker
from rasa_sdk.executor import CollectingDispatcher

# In-memory appointment storage (for demo)
appointments = {}

class ActionBookAppointment(Action):

    def name(self) -> str:
        return "action_book_appointment"

    def run(self, dispatcher: CollectingDispatcher,
            tracker: Tracker,
            domain: dict):

        user = tracker.sender_id
        date = tracker.get_slot("date")
        time = tracker.get_slot("time")

        if not date:
            dispatcher.utter_message(text="Please provide a date for the appointment.")
            return []
        if not time:
            dispatcher.utter_message(text="Please provide a time for the appointment.")
            return []

        appointments[user] = {"date": date, "time": time}
        dispatcher.utter_message(text=f"Your appointment is booked for {date} at {time}.")
        return []

class ActionCancelAppointment(Action):

    def name(self) -> str:
        return "action_cancel_appointment"

    def run(self, dispatcher: CollectingDispatcher,
            tracker: Tracker,
            domain: dict):

        user = tracker.sender_id
        if user in appointments:
            del appointments[user]
            dispatcher.utter_message(text="Your appointment has been canceled.")
        else:
            dispatcher.utter_message(text="You have no appointment to cancel.")
        return []

class ActionCheckAppointment(Action):

    def name(self) -> str:
        return "action_check_appointment"

    def run(self, dispatcher: CollectingDispatcher,
            tracker: Tracker,
            domain: dict):

        user = tracker.sender_id
        if user in appointments:
            date = appointments[user]["date"]
            time = appointments[user]["time"]
            dispatcher.utter_message(text=f"You have an appointment on {date} at {time}.")
        else:
            dispatcher.utter_message(text="You have no appointment currently.")
        return []
```

---

## **9️⃣ Endpoints**

**File:** `endpoints.yml`

```yaml
action_endpoint:
  url: "http://localhost:5055/webhook"
```

---

## **🔧 10️⃣ Running the Bot**

1. Train the model:

```bash
rasa train
```

2. Run the action server:

```bash
rasa run actions
```

3. Run the RASA server:

```bash
rasa shell
```

**Test Conversation Example:**

```
You: hi
Bot: Hi! I can help you book, check, or cancel appointments.

You: I want to book an appointment
Bot: Which date would you like to book the appointment for?

You: 2025-10-28
Bot: What time would you prefer?

You: 3 PM
Bot: Your appointment is booked for 2025-10-28 at 3 PM.

You: Check my appointment
Bot: You have an appointment on 2025-10-28 at 3 PM.

You: Cancel my appointment
Bot: Your appointment has been canceled.

You: bye
Bot: Goodbye! Have a great day.
```

---

## ✅ **11️⃣ Assignment**

1. Store appointments in **SQLite database** instead of memory.
2. Add **validation** for date and time format.
3. Integrate **Telegram** or **Streamlit** frontend.
4. Add **appointment reminders** using scheduled actions.
5. Extend bot to handle **multiple users simultaneously**.

---

I can also create a **ready-to-run version of this Appointment Bot for Telegram integration** next, similar to the Weather Bot, with **credentials.yml, ngrok setup, and action server**.

Do you want me to do that?


Perfect! Let’s create a **full-featured Appointment Booking Bot** with RASA + SQLite + Streamlit + scheduled reminders, supporting **multiple users** and **date/time validation**. I’ll break it down step by step with **ready-to-run code**.

---

# 📅 **Advanced Appointment Booking Bot**

---

## **1️⃣ Project Setup**

**Folder Structure:**

```
appointment_bot/
│
├── actions/
│   └── actions.py
├── data/
│   ├── nlu.yml
│   ├── stories.yml
│   └── rules.yml
├── config.yml
├── domain.yml
├── credentials.yml
├── endpoints.yml
├── requirements.txt
└── app.py            # Streamlit frontend
```

**requirements.txt**

```txt
rasa
rasa[spacy]
spacy
requests
schedule
sqlite3
streamlit
python-dateutil
```

Install dependencies:

```bash
python -m venv rasa_env
source rasa_env/bin/activate     # Linux/Mac
rasa_env\Scripts\activate        # Windows

pip install -r requirements.txt
python -m spacy download en_core_web_sm
```

---

## **2️⃣ SQLite Database Setup**

We'll use a database to **store appointments for multiple users**.

**File:** `actions/db.py`

```python
import sqlite3

def init_db():
    conn = sqlite3.connect('appointments.db')
    c = conn.cursor()
    c.execute('''
        CREATE TABLE IF NOT EXISTS appointments (
            user_id TEXT,
            date TEXT,
            time TEXT
        )
    ''')
    conn.commit()
    conn.close()

def add_appointment(user_id, date, time):
    conn = sqlite3.connect('appointments.db')
    c = conn.cursor()
    c.execute("INSERT INTO appointments (user_id, date, time) VALUES (?, ?, ?)", (user_id, date, time))
    conn.commit()
    conn.close()

def get_appointment(user_id):
    conn = sqlite3.connect('appointments.db')
    c = conn.cursor()
    c.execute("SELECT date, time FROM appointments WHERE user_id=?", (user_id,))
    result = c.fetchone()
    conn.close()
    return result

def delete_appointment(user_id):
    conn = sqlite3.connect('appointments.db')
    c = conn.cursor()
    c.execute("DELETE FROM appointments WHERE user_id=?", (user_id,))
    conn.commit()
    conn.close()
```

---

## **3️⃣ Custom Actions with Validation & SQLite**

**File:** `actions/actions.py`

```python
from rasa_sdk import Action, Tracker
from rasa_sdk.executor import CollectingDispatcher
from dateutil.parser import parse, ParserError
import schedule
import time
from threading import Thread
from .db import init_db, add_appointment, get_appointment, delete_appointment

init_db()

# Validate date/time
def validate_datetime(date_text, time_text):
    try:
        dt = parse(f"{date_text} {time_text}")
        return dt
    except ParserError:
        return None

# Background reminder scheduler
def reminder_scheduler():
    while True:
        schedule.run_pending()
        time.sleep(60)  # check every 60 seconds

thread = Thread(target=reminder_scheduler)
thread.daemon = True
thread.start()

class ActionBookAppointment(Action):
    def name(self) -> str:
        return "action_book_appointment"

    def run(self, dispatcher: CollectingDispatcher,
            tracker: Tracker,
            domain: dict):

        user_id = tracker.sender_id
        date = tracker.get_slot("date")
        time_slot = tracker.get_slot("time")

        if not date or not time_slot:
            dispatcher.utter_message(text="Please provide both date and time for the appointment.")
            return []

        dt = validate_datetime(date, time_slot)
        if not dt:
            dispatcher.utter_message(text="Invalid date or time format. Please try again (e.g., 2025-10-30, 3 PM).")
            return []

        add_appointment(user_id, date, time_slot)
        dispatcher.utter_message(text=f"Your appointment is booked for {date} at {time_slot}.")

        # Schedule reminder 1 hour before appointment
        schedule_time = dt.timestamp() - 3600  # seconds
        schedule.every().day.at(dt.strftime("%H:%M")).do(lambda: print(f"Reminder for user {user_id}: Appointment at {date} {time_slot}"))

        return []

class ActionCancelAppointment(Action):
    def name(self) -> str:
        return "action_cancel_appointment"

    def run(self, dispatcher: CollectingDispatcher,
            tracker: Tracker,
            domain: dict):

        user_id = tracker.sender_id
        if get_appointment(user_id):
            delete_appointment(user_id)
            dispatcher.utter_message(text="Your appointment has been canceled.")
        else:
            dispatcher.utter_message(text="You have no appointment to cancel.")
        return []

class ActionCheckAppointment(Action):
    def name(self) -> str:
        return "action_check_appointment"

    def run(self, dispatcher: CollectingDispatcher,
            tracker: Tracker,
            domain: dict):

        user_id = tracker.sender_id
        appt = get_appointment(user_id)
        if appt:
            dispatcher.utter_message(text=f"You have an appointment on {appt[0]} at {appt[1]}.")
        else:
            dispatcher.utter_message(text="You have no appointment currently.")
        return []
```

---

## **4️⃣ RASA Domain**

**File:** `domain.yml`

```yaml
version: "3.1"

intents:
  - greet
  - goodbye
  - book_appointment
  - cancel_appointment
  - check_appointment

entities:
  - date
  - time

slots:
  date:
    type: text
  time:
    type: text

responses:
  utter_greet:
    - text: "Hi! I can help you book, check, or cancel appointments."

  utter_goodbye:
    - text: "Goodbye! Have a great day."

  utter_ask_date:
    - text: "Which date would you like to book the appointment for?"

  utter_ask_time:
    - text: "What time would you prefer?"

actions:
  - action_book_appointment
  - action_cancel_appointment
  - action_check_appointment
```

---

## **5️⃣ Streamlit Frontend**

**File:** `app.py`

```python
import streamlit as st
import requests

st.set_page_config(page_title="Appointment Bot", page_icon="📅", layout="centered")
st.title("📅 Appointment Booking Bot")

if "messages" not in st.session_state:
    st.session_state.messages = []

def send_message(message):
    response = requests.post(
        "http://localhost:5005/webhooks/rest/webhook",
        json={"sender": "user", "message": message},
    )
    return response.json()

user_input = st.chat_input("Type your message...")

if user_input:
    st.session_state.messages.append({"role": "user", "content": user_input})
    responses = send_message(user_input)
    for res in responses:
        st.session_state.messages.append({"role": "assistant", "content": res.get("text")})

for msg in st.session_state.messages:
    if msg["role"] == "user":
        with st.chat_message("user"):
            st.markdown(msg["content"])
    else:
        with st.chat_message("assistant"):
            st.markdown(msg["content"])
```

---

## **6️⃣ Running the Bot**

1. Train RASA:

```bash
rasa train
```

2. Run action server:

```bash
rasa run actions
```

3. Run RASA server with API enabled:

```bash
rasa run --enable-api --cors "*"
```

4. Run Streamlit:

```bash
streamlit run app.py
```

✅ Open browser: `http://localhost:8501`

---

## **7️⃣ Features Implemented**

* Appointments stored in **SQLite**
* Validates **date and time format**
* Streamlit **frontend chat interface**
* **Scheduled reminders** (prints reminder in console)
* Supports **multiple users simultaneously** using `sender_id`

---

If you want, I can **upgrade this bot to send actual reminders via Telegram or email** instead of just printing in console.

Do you want me to do that next?
